In [81]:
import subprocess
import os
import shutil
import os
import zipfile
import random


def generate_mixed_case_string(s):
    result = "".join(random.choice([c.upper(), c.lower()]) for c in s)
    return result


def zip_folder(folder_path, zip_file_name):
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(zip_file_name, "w", zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the directory
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create the complete filepath of file in directory
                file_path = os.path.join(root, file)
                # Add file to zip
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

In [82]:
task_name = "class_data_3.3.1"
task_type = "file"  # "function"

solution_text = r"""
filename = input()
with open(filename, "r") as file:
    for i in range(8):
        print(file.readline(), end="")
"""


legend = r"""Напишите программу, которая выводит на экран первые восемь строк содержимого файла, имя которого необходимо прочитать из терминала (например, `task.txt`). Имя файла заранее не известно."""

input_info = r"\verb|str|"
output_info = r"8 \verb|str|"
additional_notes = r"""Гарантируется, что в файле найдутся 8 строк."""

test_cases = [
    "Kipling.txt",
    "Rilke.txt",
    "Esenin.txt",
    "VariableUsesOfKg.md",
    "Pasternak.md",
]

if task_type == "file+out":
    output_filename = "somefile.txt"

In [83]:
# Копируем папку
dirname = task_name
src_dir = "./sample"
dst_dir = "./" + dirname
try:
    shutil.rmtree(dst_dir)
    os.remove(f"./{task_name}.zip")
except FileNotFoundError:
    pass
shutil.copytree(src_dir, dst_dir)

'./class_data_3.3.1'

In [84]:
# Поменять ./new_task/solutions/main.py


with open(f"./{dirname}/solutions/main.py", mode="w", encoding="utf-8") as file:
    file.write(solution_text)

In [85]:
# Сгенерить тесты


# TODO: LLM-кой сделать тесты
def create_test_cases() -> list[str]:
    pass


n_tests = len(test_cases)
for i in range(n_tests):
    with open(f"./{dirname}/tests/{i + 1:02}", mode="w", encoding="utf-8") as file_test:
        file_test.write(test_cases[i])

In [86]:
# Сгенерить ответы

if task_type == "function":
    # здесь название файла-решения participantSolution, чтобы скрипт run_tests работал на стороне контеста
    with open(f"./{dirname}/files/participantSolution.py", mode="w", encoding="utf-8") as file:
        file.write(solution_text)
    commands = [
        f"python ./{dirname}/files/run_tests.py < ./{dirname}/tests/{test_number:02} > ./{dirname}/tests/{test_number:02}.a"
        for test_number in range(1, n_tests + 1)
    ]

    for cmd in commands:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True)

    subprocess.run(f"del ./{dirname}/files/participantSolution.py", shell=True, text=True, capture_output=True)
elif "file" in task_type:
    with open(f"./{dirname}/data/main.py", mode="w", encoding="utf-8") as file:
        file.write(solution_text)

    commands = [
        f"cd ./{dirname}/data/ && python ./main.py < ../tests/{test_number:02} > ../tests/{test_number:02}.a"
        for test_number in range(1, n_tests + 1)
    ]
    for cmd in commands:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True)
        print(process)

    subprocess.run(f"cd ./{dirname}/data/ && del ./main.py", shell=True, text=True, capture_output=True)
else:
    # basic prog task
    commands = [
        f"python ./{dirname}/solutions/main.py < ./{dirname}/tests/{test_number:02} > ./{dirname}/tests/{test_number:02}.a"
        for test_number in range(1, n_tests + 1)
    ]
    for cmd in commands:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True)
        print(process)

CompletedProcess(args='cd ./class_data_3.3.1/data/ && python ./main.py < ../tests/01 > ../tests/01.a', returncode=0, stdout='', stderr='')
CompletedProcess(args='cd ./class_data_3.3.1/data/ && python ./main.py < ../tests/02 > ../tests/02.a', returncode=0, stdout='', stderr='')
CompletedProcess(args='cd ./class_data_3.3.1/data/ && python ./main.py < ../tests/03 > ../tests/03.a', returncode=0, stdout='', stderr='')
CompletedProcess(args='cd ./class_data_3.3.1/data/ && python ./main.py < ../tests/04 > ../tests/04.a', returncode=0, stdout='', stderr='')
CompletedProcess(args='cd ./class_data_3.3.1/data/ && python ./main.py < ../tests/05 > ../tests/05.a', returncode=0, stdout='', stderr='')


In [87]:
# Копирование примера
import shutil

src_test = f"./{dirname}/tests/01"
src_ans = f"./{dirname}/tests/01.a"
dst_test = f"./{dirname}/statements/russian/example.01"
dst_ans = f"./{dirname}/statements/russian/example.01.a"
shutil.copy(src_test, dst_test)
shutil.copy(src_ans, dst_ans)

'./class_data_3.3.1/statements/russian/example.01.a'

In [88]:
# Изменяем problem.xml
with open(f"./{dirname}/problem.xml", mode="r", encoding="utf-8") as file:
    content = file.read()


content = content.replace("placeholder_task_name", task_name)
content = content.replace("placeholder_ntests", str(n_tests - 1))


if task_type == "file+out":
    content = content.replace('output-file=""', f'output-file="{output_filename}"')

with open(f"./{dirname}/problem.xml", mode="w", encoding="utf-8") as file:
    file.write(content)

In [89]:
# Поменять условие
with open(f"./{dirname}/statements/russian/problem.tex", mode="r", encoding="utf-8") as file:
    content = file.read()


content = content.replace("placeholder_legend", legend)
content = content.replace("placeholder_input", input_info)
content = content.replace("placeholder_output", output_info)
content = content.replace("placeholder_note", additional_notes)

with open(f"./{dirname}/statements/russian/problem.tex", mode="w", encoding="utf-8") as file:
    file.write(content)

In [90]:
zip_folder(f"{dst_dir}", f"{task_name}.zip")